# CHallenge 1 exploration

In [1]:
import pyspark
from IPython.core.display import display, HTML

conf = pyspark.SparkConf().setAppName('Jupyter').setMaster('spark://spark:7077')\
    .set('spark.jars.packages', 'org.postgresql:postgresql:42.2.18')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", '75')


display(HTML("<style>.container { width:100% !important; }</style>"))

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf5f2ade-7991-496f-9360-88ec75115dfd;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.18 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 280ms :: artifacts dl 7ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.2.18 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------------

In [3]:
spark.read\
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/postgres") \
    .option("dbtable", "public.weather") \
    .option("driver", "org.postgresql.Driver") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

city,currently_apparenttemperature,currently_humidity,currently_precipintensity,currently_precipprobability,currently_preciptype,currently_temperature,currently_visibility,currently_windspeed,date_time
Vienna,10.11,0.64,0.0,0.0,null,10.11,12.63,4.4,2019-03-28 08:05:10
Vienna,2.79,0.77,0.0178,0.05,rain,6.0,6.55,4.56,2019-03-27 05:05:11
Brussels,0.26,1.0,0.0,0.0,null,2.44,7.07,2.13,2019-03-27 05:05:05
Vienna,0.67,0.67,0.0,0.0,null,5.03,9.98,6.68,2019-03-26 05:05:10
Gothenburg,-2.12,0.88,0.0,0.0,null,0.21,10.12,1.92,2019-03-26 05:05:05
Berlin,-3.64,0.97,0.0,0.0,null,0.52,10.12,4.0,2019-03-26 05:05:04
Berlin,5.03,1.0,0.0,0.0,null,5.03,0.48,1.32,2019-03-23 06:05:04
Lyon,15.35,0.44,0.0,0.0,null,15.35,10.12,0.31,2019-03-22 11:05:07
Gothenburg,-5.59,0.87,0.0,0.0,null,-0.46,10.12,5.18,2019-03-19 21:05:07
Brussels,4.26,0.65,0.193,0.14,rain,7.19,14.68,4.6,2019-03-17 16:05:04


In [ ]:
spark.read\
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/postgres") \
    .option("dbtable", "public.weather") \
    .option("driver", "org.postgresql.Driver") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

### Analyze weather data

In [102]:
def remove_struct_name(df):
    columns = df.columns
    columns.remove('_corrupt_record')
    clean_columns = [f'`{column}` as {column.split(".")[1]}' for column in columns]
    return df.selectExpr(*clean_columns)

weather_df = spark.read.json('file:///data/weather.json').dropDuplicates()
weather_df = remove_struct_name(weather_df)

weather_df.count()

17154

In [3]:
import pyspark.sql.functions as f

weather_df\
.withColumn('currently_visibility', f.col('currently_visibility').cast('double'))\
.withColumn('currently_precipprobability', f.col('currently_precipprobability').cast('double'))\
.withColumn("valid_data", f.expr("""CASE WHEN 
city == "test" 
OR TO_TIMESTAMP(date_time, "yyyy-MM-dd HH:mm:ss") IS  NULL
OR currently_visibility NOT BETWEEN 0 AND 30
OR currently_windspeed NOT BETWEEN 0 AND 100
OR currently_humidity NOT BETWEEN 0 AND 1
OR currently_precipprobability NOT BETWEEN 0 AND 1
OR currently_precipintensity NOT BETWEEN 0 AND 10
OR currently_temperature NOT BETWEEN -50 AND 70
OR currently_apparenttemperature NOT BETWEEN -50 AND 70
OR currently_preciptype NOT IN (null, "sleet", "rain", "snow")
THEN false ELSE TRUE END"""))\
    .where('valid_data = true')\
    .withColumn('date_time', f.to_timestamp(f.col('date_time'), "yyyy-MM-dd HH:mm:ss"))\
    .registerTempTable('a')


In [8]:
spark.sql('select * from a').drop('valid_data').write.mode('append') \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/postgres") \
    .option("dbtable", "public.dude") \
    .option("driver", "org.postgresql.Driver") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .save()

In [12]:
spark.read\
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/postgres") \
    .option("dbtable", "public.dude") \
    .option("driver", "org.postgresql.Driver") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load()

city,currently_apparenttemperature,currently_humidity,currently_precipintensity,currently_precipprobability,currently_preciptype,currently_temperature,currently_visibility,currently_windspeed,date_time
Gothenburg,10.44,0.66,0.0,0.0,null,10.44,10.12,8.0,2019-03-29 16:05:05
Brussels,16.08,0.53,0.0,0.0,null,16.08,10.12,2.46,2019-03-29 14:05:05
Gothenburg,4.01,0.79,0.0,0.0,null,7.81,9.64,7.34,2019-03-29 10:05:05
Lyon,13.62,0.39,0.0,0.0,null,13.62,10.44,6.3,2019-03-27 19:05:06
Gothenburg,0.63,0.97,0.0051,0.03,rain,4.71,2.57,5.77,2019-03-27 05:05:06
Vienna,13.64,0.56,0.0,0.0,null,13.64,10.07,2.84,2019-03-24 21:05:11
Lyon,11.19,0.42,0.0,0.0,null,11.19,10.14,4.9,2019-03-20 11:05:06
Zaragoza,5.68,0.54,0.0,0.0,null,9.46,10.14,9.17,2019-03-19 20:05:10
Zaragoza,4.08,0.72,0.0914,0.09,rain,8.17,9.58,8.79,2019-03-19 03:05:10
Vienna,3.47,0.56,0.0,0.0,null,6.96,10.12,5.75,2019-03-18 22:05:10


In [10]:
# 17154
spark.sql('select* from a').printSchema()

root
 |-- city: string (nullable = true)
 |-- currently_apparenttemperature: double (nullable = true)
 |-- currently_humidity: double (nullable = true)
 |-- currently_precipintensity: double (nullable = true)
 |-- currently_precipprobability: double (nullable = true)
 |-- currently_preciptype: string (nullable = true)
 |-- currently_temperature: double (nullable = true)
 |-- currently_visibility: double (nullable = true)
 |-- currently_windspeed: double (nullable = true)
 |-- date_time: timestamp (nullable = true)
 |-- valid_data: boolean (nullable = false)



In [4]:
spark.sql('select distinct currently_preciptype from a')

currently_preciptype
null
rain
sleet
snow


In [99]:
cols = ['currently_apparenttemperature', 'currently_humidity', 'currently_precipintensity', 'currently_precipprobability', 'currently_preciptype', 'currently_temperature', 'currently_visibility', 'currently_windspeed']
min_max = ','.join([f'min({col}), max({col})' for col in cols])
spark.sql(f'select {min_max} from a')

min(currently_apparenttemperature),max(currently_apparenttemperature),min(currently_humidity),max(currently_humidity),min(currently_precipintensity),max(currently_precipintensity),min(currently_precipprobability),max(currently_precipprobability),min(currently_preciptype),max(currently_preciptype),min(currently_temperature),max(currently_temperature),min(currently_visibility),max(currently_visibility),min(currently_windspeed),max(currently_windspeed)
-22.31,34.34,0.11,1.0,0.0,6.2916,0.0,0.89,rain,snow,-5.54,34.34,0.0,16.09,0.01,33.16


In [48]:

spark.sql(f'select * from a where currently_visibility=9.58 AND currently_windspeed = 7.1 --  date_time = "2019-06-03"')

city,currently_apparenttemperature,currently_humidity,currently_precipintensity,currently_precipprobability,currently_preciptype,currently_temperature,currently_visibility,currently_windspeed,date_time
Berlin,22.6,0.66,0.3378,0.1,rain,22.57,9.58,7.1,null


In [33]:
spark.sql(f'select * from a where currently_visibility = "test"') .printSchema()

root
 |-- city: string (nullable = true)
 |-- currently_apparenttemperature: double (nullable = true)
 |-- currently_humidity: double (nullable = true)
 |-- currently_precipintensity: double (nullable = true)
 |-- currently_precipprobability: double (nullable = true)
 |-- currently_preciptype: string (nullable = true)
 |-- currently_temperature: double (nullable = true)
 |-- currently_visibility: string (nullable = true)
 |-- currently_windspeed: double (nullable = true)
 |-- date_time: string (nullable = true)



In [50]:
spark.sql(f'select currently_apparenttemperature from a order by currently_apparenttemperature desc')

currently_apparenttemperature
34.34
34.19
34.01
33.98
33.84
33.55
33.44
33.39
33.26
33.11


In [41]:
spark.sql(f'select currently_visibility from a order by currently_visibility desc')

currently_visibility
16.09
16.09
16.09
16.09
16.09
16.09
16.09
16.09
16.09
16.09


In [39]:
spark.sql(f'select currently_windspeed from a order by currently_windspeed desc')

currently_windspeed
1004.29
33.16
31.41
31.36
26.8
24.5
24.32
23.91
22.89
22.87


In [ ]:
col = 'currently_humidity'

spark.sql(f'select min({col}), max({col}) from a')

### Check track data

In [55]:
spark.sql('select to_timestamp("2019-05-13 14:23:38", "yyyy-MM-dd HH:mm:ss")')

"to_timestamp(2019-05-13 14:23:38, yyyy-MM-dd HH:mm:ss)"
2019-05-13 14:23:38


In [51]:
events_df = spark.read.json('file:///data/track_events.json')
events_df = remove_struct_name(events_df).dropDuplicates()\
    .withColumn("context_os_name", f.expr("""CASE 
            WHEN  context_os_name NOT IN (null, "Android","iOS") THEN 
                CASE WHEN context_device_type = "android" THEN "Android" 
                WHEN context_device_type = "IOS" THEN "iOS" END
            ELSE context_os_name END"""))\
    .withColumn("valid_data", f.expr("""CASE WHEN 
        TO_TIMESTAMP(received_time, "yyyy-MM-dd HH:mm:ss") IS  NULL
        OR TO_TIMESTAMP(sent_time, "yyyy-MM-dd HH:mm:ss") IS  NULL
        OR TO_TIMESTAMP(original_timestamp_time, "yyyy-MM-dd HH:mm:ss") IS  NULL
        OR BIGINT(REPLACE(context_app_version, '.','')) IS NULL
        OR BIGINT(REPLACE(context_os_version, '.','')) IS NULL
        OR BIGINT(REPLACE(context_app_build, '.','')) IS NULL
        OR anonymous_id IS NULL
        OR event_name IS NULL
        THEN false ELSE TRUE END"""))
                
events_df.drop('_corrupt_record').withColumn('received_time', f.to_timestamp(f.col('received_time'), "yyyy-MM-dd HH:mm:ss"))\
    .withColumn('sent_time', f.to_timestamp(f.col('sent_time'), "yyyy-MM-dd HH:mm:ss"))\
    .withColumn('original_timestamp_time', f.to_timestamp(f.col('original_timestamp_time'), "yyyy-MM-dd HH:mm:ss"))\
.registerTempTable('b')

In [54]:
spark.sql('select * from b where event_name is null')

anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time,valid_data
null,null,null,null,206A4930-3248-46A8-94D7-60DDD435B05D,Apple,"iPhone10,5",ios,iOS,12.0.1,Europe/Oslo,null,null,null,null,2019-05-13 14:23:38,false
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false


In [53]:
spark.sql('select * from b where anonymous_id is null')

anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time,valid_data
null,null,null,null,206A4930-3248-46A8-94D7-60DDD435B05D,Apple,"iPhone10,5",ios,iOS,12.0.1,Europe/Oslo,null,null,null,null,2019-05-13 14:23:38,false
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,false


In [17]:
spark.sql('select * from b').printSchema()

root
 |-- anonymous_id: string (nullable = true)
 |-- context_app_build: string (nullable = true)
 |-- context_app_name: string (nullable = true)
 |-- context_app_version: string (nullable = true)
 |-- context_device_id: string (nullable = true)
 |-- context_device_manufacturer: string (nullable = true)
 |-- context_device_model: string (nullable = true)
 |-- context_device_type: string (nullable = true)
 |-- context_os_name: string (nullable = true)
 |-- context_os_version: string (nullable = true)
 |-- context_timezone: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- original_timestamp_time: string (nullable = true)
 |-- properties_rating: string (nullable = true)
 |-- received_time: string (nullable = true)
 |-- sent_time: string (nullable = true)



In [21]:
spark.sql('select distinct initcap(context_device_manufacturer) as gg from b order by gg')

gg
null
A-gold
Acer
Agm
Alps
Apple
Asus
Blackberry
Blackview
Blu


In [113]:
spark.sql('select distinct context_app_name from b')

context_app_name
null
Tier


In [19]:
spark.sql('select distinct context_device_type from b')

context_device_type
null
android
ios


In [106]:
spark.sql('select distinct context_os_name from b')

context_os_name
null
""
Android
iOS


In [116]:
spark.sql('select distinct event_name from b order by event_name asc')

event_name
null
Account Setup Opened
App Rated - Feedback
App Rated - Forwarded
App Rated - In App
App Rated - No App Store
Application Installed
Application Opened
Application Updated
Customer Blocked - Phone Validation


In [117]:
spark.sql('select distinct properties_rating from b')

properties_rating
null
3
1
5
4
2


In [114]:
spark.sql('select distinct context_app_version from b')

context_app_version
null
3.0.2
3.0.3
3.0.7
3.0.1
3.0.6
3.0.8
3.0.0
3.0.5
3.0.4


In [118]:
spark.sql('select distinct context_device_model from b')

context_device_model
Phantom6-Plus
HTC U11 life
SM-A505FN
SM-J710F
SM-A750F
Redmi Note 4
LG-H815
CLT-L04
SM-J530F
SM-T580


In [28]:
spark.sql('select distinct context_timezone from b order by context_timezone desc') 

context_timezone
Pacific/Noumea
Pacific/Midway
Europe/Zurich
Europe/Zagreb
Europe/Warsaw
Europe/Vilnius
Europe/Vienna
Europe/Vatican
Europe/Tallinn
Europe/Stockholm


In [22]:
spark.sql('select context_os_version from b order by context_os_version desc') 

context_os_version
Q
Q
Q
Q
Q
9
9
9
9
9


In [23]:
spark.sql('select * from b where event_name is null')

anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,206A4930-3248-46A8-94D7-60DDD435B05D,Apple,"iPhone10,5",ios,iOS,12.0.1,Europe/Oslo,null,null,null,null,2019-05-13 14:23:38


In [30]:
spark.sql('select * from b WHERE context_os_name ="iOS" and context_device_type = "android"')

anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time


In [107]:
spark.sql('select * from b WHERE context_os_name =""')

_corrupt_record,anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time
null,476D8356-AEC5-40D6-96FE-1D5E06D22ACB,3.0.4.1,Tier,3.0.4,3932FD8C-BD82-4240-8AD6-3F2369705EF5,Apple,"iPhone7,2",ios,,12.2,Europe/Copenhagen,Signup Started,2019-05-13 14:23:45,null,2019-05-13 14:23:47,2019-05-13 14:23:47
null,3aeb3f30-416c-4255-a44d-ce15e4938bfb,300380,Tier,3.0.8,d46c3ede716c676c,Xiaomi,Redmi 5,android,,7.1.2,Europe/Madrid,Map View Opened,2019-05-13 14:23:44,null,2019-05-13 14:23:47,2019-05-13 14:23:47


In [26]:
spark.sql('select * from b where context_os_version = "Q"')

anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time
828a755b-6e5f-4b3c-ae28-7cdfac2fa9bf,300380,Tier,3.0.8,420cc7c499b4b243,Google,Pixel 2,android,Android,Q,Europe/Berlin,Application Opened,2019-05-13 11:57:05,null,2019-05-13 11:57:36,2019-05-13 11:57:36
828a755b-6e5f-4b3c-ae28-7cdfac2fa9bf,300380,Tier,3.0.8,420cc7c499b4b243,Google,Pixel 2,android,Android,Q,Europe/Berlin,Application Updated,2019-05-13 11:57:05,null,2019-05-13 11:57:36,2019-05-13 11:57:36
828a755b-6e5f-4b3c-ae28-7cdfac2fa9bf,300380,Tier,3.0.8,420cc7c499b4b243,Google,Pixel 2,android,Android,Q,Europe/Berlin,Scooter Selected - User,2019-05-13 11:57:19,null,2019-05-13 11:57:36,2019-05-13 11:57:36
828a755b-6e5f-4b3c-ae28-7cdfac2fa9bf,300380,Tier,3.0.8,420cc7c499b4b243,Google,Pixel 2,android,Android,Q,Europe/Berlin,Map View Opened,2019-05-13 11:57:07,null,2019-05-13 11:57:36,2019-05-13 11:57:36
828a755b-6e5f-4b3c-ae28-7cdfac2fa9bf,300380,Tier,3.0.8,420cc7c499b4b243,Google,Pixel 2,android,Android,Q,Europe/Berlin,Scooter Selected - User,2019-05-13 11:57:31,null,2019-05-13 11:57:36,2019-05-13 11:57:36


In [119]:
spark.sql('select * from b where context_app_name is null')

_corrupt_record,anonymous_id,context_app_build,context_app_name,context_app_version,context_device_id,context_device_manufacturer,context_device_model,context_device_type,context_os_name,context_os_version,context_timezone,event_name,original_timestamp_time,properties_rating,received_time,sent_time
"{""segment_tracks.original_timestamp_time"":""2019-05-13 14:23:48"",""segment...",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,206A4930-3248-46A8-94D7-60DDD435B05D,Apple,"iPhone10,5",ios,iOS,12.0.1,Europe/Oslo,null,null,null,null,2019-05-13 14:23:38
